This notebook is a simple example how to use DeGirum PySDK to do AI inference of a video stream from local camera.
OpenCV is required to run this sample.

In [ ]:
import degirum as dg # import DeGirum PySDK
import cv2 # OpenCV

In [ ]:
import gradio as gr

In [ ]:
# connect to default model zoo
zoo = dg.connect_model_zoo('192.168.0.169')
# load AI model 'ssd_mobilenet_v2' for DeGirum Orca AI accelerator
model = zoo.load_model("mobilenet_v1_imagenet--224x224_quant_n2x_cpu_1")

# set model parameters
model.image_backend = 'pil' # select OpenCV backend: needed to have overlay image in OpenCV format
model.overlay_show_probabilities = True # show probabilities on overlay image
model.overlay_font_scale=4

In [ ]:
orca_models=zoo.list_models(device='orca')

In [5]:
orca_models

['efficientdet_lite1_coco--384x384_quant_n2x_orca_1',
 'efficientnet_em_imagenet--240x240_quant_n2x_orca_1',
 'efficientnet_es_imagenet--224x224_quant_n2x_orca_1',
 'mobiledet_coco--320x320_quant_n2x_orca_1',
 'mobilenet_v1_imagenet--224x224_quant_n2x_orca_1',
 'mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca_1',
 'mobilenet_v1_yamnet_sound_cls--96x64_quant_n2x_orca_1',
 'mobilenet_v2_generic_object--224x224_quant_n2x_orca_1',
 'mobilenet_v2_imagenet--224x224_float_n2x_orca_1',
 'mobilenet_v2_imagenet--224x224_quant_n2x_orca_1',
 'mobilenet_v2_sign_language--224x224_float_n2x_orca_1',
 'mobilenet_v2_ssd_coco--300x300_quant_n2x_orca_1',
 'mobilenet_v2_ssd_generic_object--192x192_quant_n2x_orca_1',
 'resnet50_imagenet--224x224_pruned_quant_n2x_orca_1',
 'yolo_v5m_coco--512x512_pruned_quant_n2x_orca_1',
 'yolo_v5s_coco--512x512_quant_n2x_orca_1',
 'yolo_v5s_coco--576x576_quant_n2x_orca_1',
 'yolo_v5s_face_det--512x512_quant_n2x_orca_1',
 'yolo_v5s_hand_det--512x512_quant_n2x_o

In [ ]:
pose_model=zoo.load_model("mobilenet_v1_posenet_coco_keypoints--353x481_quant_n2x_orca_1")
pose_model.overlay_show_labels=False
pose_model.overlay_show_labels = False
pose_model.overlay_line_width = 2
pose_model.overlay_alpha = 1
pose_model.image_backend='pil'


In [ ]:
def posenet(test_image):
    poses=pose_model(test_image)
    faces=model(test_image)
    return poses.image_overlay, faces.image_overlay

video_input=gr.inputs.Video(    source="webcam", 
                                label=None, 
                                optional=False)
image_input=gr.inputs.Image(    shape=None, 
                                image_mode="RGB", 
                                invert_colors=False, 
                                source="upload", 
                                tool="editor", 
                                type="numpy", 
                                label=None, 
                                optional=False)

iface = gr.Interface(fn=posenet, inputs=video_input, outputs=["image","image"])
iface.launch()

In [ ]:
def posenet(test_image,model_name):
    model=zoo.load_model(model_name)
    image_list=[test_image for i in range(0,1)]
    for poses in model.predict_batch(image_list):
        pass
    return poses.image_overlay

image_input=gr.inputs.Image(    shape=None, 
                                image_mode="RGB", 
                                invert_colors=False, 
                                source="upload", 
                                tool="editor", 
                                type="numpy", 
                                label=None, 
                                optional=False)
model_name=gr.inputs.Dropdown(orca_models, type="value", default=None, label=None, optional=False)


iface = gr.Interface(fn=posenet, inputs=[image_input,model_name], outputs=["image"])
iface.launch()

In [ ]:
def posenet(test_image,model_name):
    model=zoo.load_model(model_name)
    image_list=[test_image for i in range(0,1)]
    for poses in model.predict_batch(image_list):
        pass
    return poses

image_input=gr.inputs.Image(    shape=None, 
                                image_mode="RGB", 
                                invert_colors=False, 
                                source="upload", 
                                tool="editor", 
                                type="numpy", 
                                label=None, 
                                optional=False)
model_name=gr.inputs.Dropdown(orca_models, type="value", default=None, label=None, optional=False)


iface = gr.Interface(fn=posenet, inputs=[image_input,model_name], outputs=["textbox"])
iface.launch()

In [ ]:
pose_model.measure_time=True

In [ ]:
pose_model.time_stats()['FrameTotalDuration_ms'].avg

In [ ]:
import gradio as gr

model1 = zoo.load_model("mobilenet_v1_imagenet--224x224_quant_n2x_orca_1")
model2 = zoo.load_model("mobilenet_v2_imagenet--224x224_quant_n2x_orca_1")

gr.Parallel(model1, model2).launch()

In [ ]:
stream = cv2.VideoCapture(1)

# define iterator function, which returns frames from camera 
def source():
    while True:
        ret, frame = stream.read()
        yield frame

# AI prediction loop
# Press 'x' to stop
for res in pose_model.predict_batch(source()):
    cv2.imshow("AI camera", res.image_overlay)
    if (cv2.waitKey(1) & 0xFF) == ord('x'):
        break
        
stream.release() # release camera stream
cv2.destroyAllWindows() # close OpenCV windows

In [ ]:
def classify_image(inp):
    print(model(inp).results)
    return model(inp).results

image = gr.inputs.Image(label="Input Image", source="upload")
label = gr.outputs.Textbox()

gr.Interface(fn=classify_image, inputs=image, outputs=label).launch()

In [ ]:
import numpy as np

import gradio as gr


def snap(image):
    return np.flipud(image)


iface = gr.Interface(snap, gr.inputs.Video(source="webcam"), "image")
iface.launch()

In [ ]:
pose_model.input_numpy_colorspace

In [ ]:
dg.__version__

In [ ]:
# open video stream from local camera #0
stream = cv2.VideoCapture(1)

# define iterator function, which returns frames from camera 
def source():
    while True:
        ret, frame = stream.read()
        yield frame

# AI prediction loop
# Press 'x' to stop
for res in pose_model.predict_batch(source()):
    cv2.imshow("AI camera", res.image_overlay)
    if (cv2.waitKey(1) & 0xFF) == ord('x'):
        break
        
stream.release() # release camera stream
cv2.destroyAllWindows() # close OpenCV windows

In [ ]:
zoo.list_models()